# DATA PREPROCESSING


In [97]:
# DATA PREPROCESSING
# MFCCs

import pandas as pd
import numpy as np
import tables  
import librosa
import librosa.display
# time taken to read data
chunk = pd.read_excel('ESC-50-master/meta/miniesc50.xlsx',usecols=['filename','category'])
classes = chunk['category']
audiofiles = chunk['filename']

mfccFeatures = []
i = 0
for wavFile in audiofiles:
    data, samplingRate = librosa.load(('ESC-50-master/audio/' + wavFile), sr=24050)

    #Grab the mfcc features through the mfcc algorithm
    mfcc = librosa.feature.mfcc(data,sr=samplingRate,n_mfcc=40)
    mfccs_scaled_features = np.mean(mfcc.T,axis=0)


    mfccFeatures.append([mfccs_scaled_features,classes[i]])
    i += 1
    if (i == 200):
        break
pd.set_option('display.max_colwidth', None)
mfccFeatures = pd.DataFrame(mfccFeatures, columns=['feature','class_category'])


C:\Users\reals\AppData\Local\Temp\ipykernel_22792\4107882624.py:20: FutureWarning: Pass y=[-0.00981905 -0.01689685 -0.00292127 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(data,sr=samplingRate,n_mfcc=40)
C:\Users\reals\AppData\Local\Temp\ipykernel_22792\4107882624.py:20: FutureWarning: Pass y=[-0.00605555 -0.00881997 -0.00775089 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(data,sr=samplingRate,n_mfcc=40)
C:\Users\reals\AppData\Local\Temp\ipykernel_22792\4107882624.py:20: FutureWarning: Pass y=[ 0.02323986  0.03524191  0.03238183 ... -0.09332374 -0.1408261
 -0.12760696] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(data,sr=samplingRate,n_mfcc=40)
C:\Users\reals\AppData\Local\Temp

# MODEL CREATION AND TESTING

In [1]:
# MODEL CREATION AND TESTING

import pandas as pd
import numpy as np
import tables  
import librosa
import librosa.display
from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, ShuffleSplit
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from ann_visualizer.visualize import ann_viz;

# from tensorflow import keras 
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from numpy import load

X = load('X.npy')

Y = load('Y.npy')

print(X.shape)

#Encode target labels with value between 0 and n-1 classes
Label_encoder = LabelEncoder()
yy=to_categorical((Label_encoder.fit_transform(Y)))


#performing train test split on our data set.
X_train,X_test,Y_train,Y_test=train_test_split(X,yy,test_size=0.40,shuffle = True, random_state=2)

num_labels=yy.shape[1]

model=Sequential()
#first layer
model.add(Dense(256,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

# To compile the model we need to define loss function which is categorical cross-entropy,
# accuracy metrics which is accuracy score, and an optimizer which is Adam.

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
model.summary()
# Calculate pre-training accuracy 
score = model.evaluate(X_test, Y_test, verbose=0)
accuracy = 400*score[1]

# TESTING BEFORE TRAINING
print(f'the accuracy {accuracy} before training')

# TRAINING
num_epochs = 200
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='./audio_classification.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(X_train, Y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, Y_test), callbacks=[checkpointer] ,verbose=1)

# EVALUATING AFTER TRAINING
score = model.evaluate(X_train, Y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))

score = model.evaluate(X_test, Y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))



# PREDICTION

(200, 40)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               10496     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 1285      
                                                                 
 activation_1 (Activation)   (None, 5)                 0         
                                                                 
Total params: 11,781
Trainable params: 11,781
Non-trainable params: 0
_________________________________________________________________
the accuracy 44.999998807907104 before tra

#PREDICTION


In [ ]:
filename="testSounds/sirening.wav"

#preprocess the audio file
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

#Reshape MFCC feature to 2-D array
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)

x_predict=model.predict(mfccs_scaled_features) 
predicted_label=np.argmax(x_predict,axis=1)

prediction_class = Label_encoder.inverse_transform(predicted_label) 
print(prediction_class)

1/1 [==============================] - 0s 185ms/step
['siren']


In [26]:
# 25 SAMPLES

import glob, os, sys 
import pandas as pd
sys.stdout.flush()

# Set the directory path and the file extension
dir_path = 'samples25'
file_extension = '*.wav'
final_ans = []
# Get a list of all the files with the specified extension
file_list = glob.glob(os.path.join(dir_path, file_extension))

for x in file_list:
    #preprocess the audio file
    audio, sample_rate = librosa.load(x, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    #Reshape MFCC feature to 2-D array
    mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)

    x_predict=model.predict(mfccs_scaled_features) 
    predicted_label=np.argmax(x_predict,axis=1)

    prediction_class = Label_encoder.inverse_transform(predicted_label) 
    final_ans.append([x,prediction_class])
    print(x,prediction_class,flush=True)
table = pd.DataFrame(final_ans, columns=['feature','class_category'])

table.to_excel('the25Samples.xlsx')
# print(final_ans,end='')

1/1 [==============================] - 0s 48ms/step
samples25\carhorn1.wav ['car_horn']
1/1 [==============================] - 0s 35ms/step
samples25\carhorn2.wav ['car_horn']
1/1 [==============================] - 0s 38ms/step
samples25\carhorn3.wav ['car_horn']
1/1 [==============================] - 0s 36ms/step
samples25\carhorn4.wav ['car_horn']
1/1 [==============================] - 0s 33ms/step
samples25\carhorn5.wav ['car_horn']
1/1 [==============================] - 0s 29ms/step
samples25\cat1.wav ['cat']
1/1 [==============================] - 0s 36ms/step
samples25\cat2.wav ['cat']
1/1 [==============================] - 0s 34ms/step
samples25\cat3.wav ['cat']
1/1 [==============================] - 0s 42ms/step
samples25\cat4.wav ['cat']
1/1 [==============================] - 0s 34ms/step
samples25\cat5.wav ['cat']
1/1 [==============================] - 0s 33ms/step
samples25\dog1.wav ['car_horn']
1/1 [==============================] - 0s 33ms/step
samples25\dog2.wav ['dog']
1/